In [50]:
import clingo

from typing import Sequence

In [51]:
def solve(programs: Sequence[str], grounding_context=None, sep:str=' '):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

In [52]:
domain = """

contract("Instanz Tauschvertrag").

component("Lieferung Ration").
component("Rückabwicklung Ration").

component("Lieferung Brett").
component("Nachfrist Lieferung Brett").
component("Lieferung Brett cont.").

component("Lieferung Brett Austauschanspruch").
component("Lieferung Brett Differenzanspruch").

componentType("Sachleistung").
componentType("Nachfrist").
componentType("Geldleistung").
componentType("Austauschanspruch").
componentType("Differenzanspruch").

componentState("Initialisiert").
componentState("Initialisiert [GLE]").
componentState("Gewährleistung [GLE]").
componentState("Abgeschlossen [GLE]").
componentState("Unmöglichkeit (Schuldner)").
componentState("Unmöglichkeit (Schuldner) [GLE]").
% componentState("Unmöglichkeit (Gläubiger)").
% componentState("Unmöglichkeit (Gläubiger) [GLE]").
% componentState("Unmöglichkeit (Zufall)").
% componentState("Unmöglichkeit (Zufall) [GLE]").
% componentState("Verzug (Schuldner)").
% componentState("Verzug (Schuldner) [GLE]").
% componentState("Verzug (Schuldner, Vertrag besteht)").
% componentState("Verzug (Schuldner, Vertrag besteht) [GLE]").
% componentState("Verzug (Schuldner, Vom Vertrag zurückgetreten)").
% componentState("Verzug (Schuldner, Vom Vertrag zurückgetreten) [GLE]").
% componentState("Verzug (Gläubiger)").
% componentState("Verzug (Gläubiger) [GLE]").


resource("Holz").
resource("Nahrung").
resource("Brett").
resource("Ration").
resource("Gemälde").
resource("Geld").

resourceType("Gattung").
resourceType("Spezies").
resourceType("Geld").

agent(bob).
agent(woody).

event(destroy(Resource)) :- resource(Resource).

action(gather("Holz")).
action(gather("Nahrung")).
action(process_to("Holz", "Brett")).
action(process_to("Nahrung", "Ration")).
action(give_to_because(Resource, Agent, Component)) :- agent(Agent), resource(Resource), component(Component).
action(accept_from_because(Resource, Agent, Component)) :- agent(Agent), resource(Resource), component(Component).

action(communicate_to_because("Am Vertrag festhalten", Agent, Component)) :- agent(Agent), component(Component).
action(communicate_to_because("Vom Vertrag zurücktreten", Agent, Component)) :- agent(Agent), component(Component).

time(0..10).

"""
solve([domain]);

Answer 1: { action(gather("Holz")) action(gather("Nahrung")) action(process_to("Holz","Brett")) action(process_to("Nahrung","Ration")) action(accept_from_because("Brett",bob,"Lieferung Brett")) action(accept_from_because("Brett",bob,"Lieferung Brett Austauschanspruch")) action(accept_from_because("Brett",bob,"Lieferung Brett Differenzanspruch")) action(accept_from_because("Brett",bob,"Lieferung Brett cont.")) action(accept_from_because("Brett",bob,"Lieferung Ration")) action(accept_from_because("Brett",bob,"Nachfrist Lieferung Brett")) action(accept_from_because("Brett",bob,"Rückabwicklung Ration")) action(accept_from_because("Brett",woody,"Lieferung Brett")) action(accept_from_because("Brett",woody,"Lieferung Brett Austauschanspruch")) action(accept_from_because("Brett",woody,"Lieferung Brett Differenzanspruch")) action(accept_from_because("Brett",woody,"Lieferung Brett cont.")) action(accept_from_because("Brett",woody,"Lieferung Ration")) action(accept_from_because("Brett",woody,"Nac

In [53]:
contract = """

contract_component("Instanz Tauschvertrag", "Lieferung Ration").
contract_component("Instanz Tauschvertrag", "Lieferung Brett").

component_type("Lieferung Ration", "Sachleistung").
component_provider("Lieferung Ration", bob).
component_beneficiary("Lieferung Ration", woody).
component_item("Lieferung Ration", "Ration").

component_type("Lieferung Brett", "Sachleistung").
component_provider("Lieferung Brett", woody).
component_beneficiary("Lieferung Brett", bob).
component_item("Lieferung Brett", "Brett").

deo_at(component_state("Lieferung Ration", "Initialisiert"), 0).
deo_at(component_state("Lieferung Brett", "Initialisiert"), 0).

"""
solve([contract]);

Answer 1: { component_beneficiary("Lieferung Brett",bob) component_beneficiary("Lieferung Ration",woody) component_item("Lieferung Brett","Brett") component_item("Lieferung Ration","Ration") component_provider("Lieferung Brett",woody) component_provider("Lieferung Ration",bob) component_type("Lieferung Brett","Sachleistung") component_type("Lieferung Ration","Sachleistung") contract_component("Instanz Tauschvertrag","Lieferung Brett") contract_component("Instanz Tauschvertrag","Lieferung Ration") deo_at(component_state("Lieferung Brett","Initialisiert"),0) deo_at(component_state("Lieferung Ration","Initialisiert"),0) }
SAT 1


In [54]:
instance = """

occ_at(agent_does(bob, gather("Nahrung")), 0).
occ_at(agent_does(woody, gather("Holz")), 0).

occ_at(agent_does(bob, process_to("Nahrung", "Ration")), 1).
occ_at(agent_does(woody, process_to("Holz", "Brett")), 1).

occ_at(agent_does(woody, destroy("Brett")), 3).

"""
solve([instance]);

Answer 1: { occ_at(agent_does(bob,gather("Nahrung")),0) occ_at(agent_does(bob,process_to("Nahrung","Ration")),1) occ_at(agent_does(woody,destroy("Brett")),3) occ_at(agent_does(woody,gather("Holz")),0) occ_at(agent_does(woody,process_to("Holz","Brett")),1) }
SAT 1


In [55]:
rules = """

component_dual(Component, Dual) :-
 component(Component), component(Dual),
 contract(Contract),
 Component != Dual,
 contract_component(Contract, Component), contract_component(Contract, Dual).

imp_at(component_state(Component, State), T) :-
  component(Component), componentState(State), time(T),
  deo_at(component_state(Component, AlternativeState), T),
  not inertial_to(component_state(Component, AlternativeState), T),
  AlternativeState != State.

inertial_to(component_state(Component, State), T+1) :-
  time(T+1), time(T),
  deo_at(component_state(Component, State), T),
  not imp_at(component_state(Component, State), T+1).

%% INERTIA
deo_at(component_state(Component, State), T) :-
  inertial_to(component_state(Component, State), T).

%%% TRANSITION
%% Transition Initialisiert -> Initialisiert [GLE]
% Dual Gewährleistung
deo_at(component_state(Component, "Initialisiert [GLE]"), T+1) :-
  component(Component), time(T+1),
  component_dual(Component, Dual),
  deo_at(component_state(Component, "Initialisiert"),            T),
  deo_at(component_state(Dual, "Gewährleistung"),                T+1),
  not imp_at(component_state(Component, "Initialisiert [GLE]"),  T+1).

% Dual Abgeschlossen
deo_at(component_state(Component, "Initialisiert [GLE]"), T+1) :-
  component(Component), time(T+1),
  component_dual(Component, Dual),
  deo_at(component_state(Component, "Initialisiert"),            T),
  deo_at(component_state(Dual, "Abgeschlossen"),                 T+1),
  not imp_at(component_state(Component, "Initialisiert [GLE]"),   T+1).

%% Transition Initialisiert -> Gewährleistung
deo_at(component_state(Component, "Gewährleistung"), T+1) :-
  component(Component), time(T+1),
  time(T),
  deo_at(component_state(Component, "Initialisiert"),            T),
  rul_component_at(transaction_from_to(_, _, _), Component,      T),
  not imp_at(component_state(Component, "Gewährleistung"),       T+1).

%% Transition Initialisiert [GLE] -> Gewährleistung [GLE]
deo_at(component_state(Component, "Gewährleistung [GLE]"), T+1) :-
  component(Component), time(T+1),
  time(T),
  deo_at(component_state(Component, "Initialisiert [GLE]"), T),
  rul_component_at(transaction_from_to(_, _, _), Component, T).

%% Transition Gewährleistung -> Gewährleistung [GLE]
% Dual Gewährleistung [GLE]
deo_at(component_state(Component, "Gewährleistung [GLE]"), T+1) :-
  component(Component), time(T+1),
  time(T), component_dual(Component, Dual),
  deo_at(component_state(Component, "Gewährleistung"), T),
  deo_at(component_state(Dual, "Gewährleistung [GLE]"), T).

% Dual Abgeschlossen [GLE]
deo_at(component_state(Component, "Gewährleistung [GLE]"), T+1) :-
  component(Component), time(T+1),
  time(T), component_dual(Component, Dual),
  deo_at(component_state(Component, "Gewährleistung"), T),
  deo_at(component_state(Dual, "Abgeschlossen [GLE]"), T).

%% Transition Any -> Unmöglichkeit (Schuldner)
deo_at(component_state(Component, "Unmöglichkeit (Schuldner)"), T+1) :-
  component(Component), time(T+1),
  time(T), component_provider(Component, Provider),
  rul_component_at(impossibility(Provider), Component, T),
  not imp_at(component_state(Component, "Unmöglichkeit (Schuldner)"), T+1).

% Physical Exchange leads to Deontic Transaction
rul_component_at(transaction_from_to(Item, Provider, Beneficiary), Component, T) :-
  resource(Item), agent(Provider), agent(Beneficiary), component(Component), time(T),
  component_provider(Component, Provider),
  component_beneficiary(Component, Beneficiary),
  component_item(Component, Item),
  occ_at(agent_does(Provider, give_to_because(Item, Beneficiary, Component)),     T),
  occ_at(agent_does(Beneficiary, accept_from_because(Item, Provider, Component)), T).

% An agent destroying an object leads to Deontic Impossiblity, caused by the agent
rul_component_at(impossibility(Agent), Component, T) :-
  agent(Agent), component(Component), time(T),
  component_item(Component, Item),
  occ_at(agent_does(Agent, destroy(Item)), T).

"""
solve([rules]);

Answer 1: {  }
SAT 1


<block>:4:2-22: info: atom does not occur in any rule head:
  component(Component)

<block>:4:24-39: info: atom does not occur in any rule head:
  component(Dual)

<block>:5:2-20: info: atom does not occur in any rule head:
  contract(Contract)

<block>:7:2-41: info: atom does not occur in any rule head:
  contract_component(Contract,Component)

<block>:7:43-77: info: atom does not occur in any rule head:
  contract_component(Contract,Dual)

<block>:10:3-23: info: atom does not occur in any rule head:
  component(Component)

<block>:10:25-46: info: atom does not occur in any rule head:
  componentState(State)

<block>:10:48-55: info: atom does not occur in any rule head:
  time(T)

<block>:16:3-12: info: atom does not occur in any rule head:
  time((T+1))

<block>:16:14-21: info: atom does not occur in any rule head:
  time(T)

<block>:28:3-23: info: atom does not occur in any rule head:
  component(Component)

<block>:28:25-34: info: atom does not occur in any rule head:
  time((T+1))

In [56]:
solve([
    domain,
    rules,
    contract,
    instance,
    '#show occ_at/2.',
    '#show obs_at/2.',
    '#show deo_at/2.',
    '#show rul_component_at/3.'
    #'#show imp_at/2.',
], sep='\n');

Answer 1: {
deo_at(component_state("Lieferung Brett","Initialisiert"),0)
deo_at(component_state("Lieferung Brett","Initialisiert"),1)
deo_at(component_state("Lieferung Brett","Initialisiert"),2)
deo_at(component_state("Lieferung Brett","Initialisiert"),3)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner)"),4)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner)"),5)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner)"),6)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner)"),7)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner)"),8)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner)"),9)
deo_at(component_state("Lieferung Brett","Unmöglichkeit (Schuldner)"),10)
deo_at(component_state("Lieferung Ration","Initialisiert"),0)
deo_at(component_state("Lieferung Ration","Initialisiert"),1)
deo_at(component_state("Lieferung Ration","Initialisiert"),2)
deo_at(component_state("Lieferung Ration","Ini

<block>:1:1-16: info: no atoms over signature occur in program:
  obs_at/2

